In [1]:
from transformers.convert_graph_to_onnx import convert_pytorch, load_graph_from_args, infer_shapes, ensure_valid_input

pipeline_name = "feature-extraction"
framework = "pt"
model = "bert-base-cased"
tokenizer = "bert-base-cased"
nlp = load_graph_from_args(pipeline_name, framework, model, tokenizer)

input_names, output_names, dynamic_axes, tokens = infer_shapes(nlp, "pt")
ordered_input_names, model_args = ensure_valid_input(nlp.model, tokens, input_names)



Loading pipeline (model: bert-base-cased, tokenizer: bert-base-cased)


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Found input input_ids with shape: {0: 'batch', 1: 'sequence'}
Found input token_type_ids with shape: {0: 'batch', 1: 'sequence'}
Found input attention_mask with shape: {0: 'batch', 1: 'sequence'}
Found output output_0 with shape: {0: 'batch', 1: 'sequence'}
Found output output_1 with shape: {0: 'batch'}
Ensuring inputs are in correct order
position_ids is not present in the generated input list.
Generated inputs order: ['input_ids', 'attention_mask', 'token_type_ids']


In [2]:
import os
import torch
from modeling_bert import BertForSequenceClassification, BertWithExit_s1, BertWithExit_s2

model = BertWithExit_s2.from_pretrained('bert-base-uncased')
model.add_exit(7,8)

dynamic_axes.pop('output_0')
dynamic_axes.pop('output_1')
dynamic_axes.pop('token_type_ids')

# output_names = ["output1", "exit_output"]
output_names = ["final_output"]

# dynamic_axes['output1'] = {0: 'batch', 1: 'sequence'}
# dynamic_axes['exit_output'] = {0: 'batch'}
dynamic_axes['final_output'] = {0: 'batch'}
model_args = (torch.Tensor(1, 7, 768), torch.Tensor(1, 7))

torch.onnx.export(
    model,
    model_args,
    f="./bert_model.onnx",
    input_names=ordered_input_names[:2],
    output_names=output_names,
    dynamic_axes=dynamic_axes,
    do_constant_folding=True,
    enable_onnx_checker=True,
    opset_version=11,
)
# torch.onnx.export(model, "./bert_model.onnx")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertWithExit_s2: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertWithExit_s2 from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertWithExit_s2 from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertWithExit_s2 were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['s2_poole

In [8]:
model_args = (torch.Tensor(1, 7, 768), torch.Tensor(1, 1, 1, 7))

In [11]:
model_args[0].shape

torch.Size([1, 7, 768])